In [2]:
!pip install datasets

In [4]:
import json
from datasets import Dataset, DatasetDict

In [3]:
filename = "/content/drive/MyDrive/AAI_project/dataset_v1.jsonl"

In [5]:
standardized_data = []

In [6]:
with open(filename, 'r') as f:
    for line in f:
        entry = json.loads(line)
        if 'output' in entry:
            if isinstance(entry['output'], (list, dict)):
                entry['output'] = json.dumps(entry['output'])
            else:
                entry['output'] = str(entry['output'])
        standardized_data.append(entry)

In [8]:
ds = Dataset.from_list(standardized_data)

In [9]:
split = ds.train_test_split(test_size=0.2, seed=42)
dd = DatasetDict({"train": split["train"], "test": split["test"]})

In [17]:
dd.save_to_disk("preprocessed_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/269 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/68 [00:00<?, ? examples/s]

In [10]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments

In [11]:
base_model = "openai-community/gpt2"

In [13]:
tok = AutoTokenizer.from_pretrained(base_model, use_fast=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [20]:
def pack(ex):
    prompt = f"""
    ### Instruction: {ex['instruction']}\n
    ### Input: {ex['input']}\n
    ### Output: {ex['output']}
    """
    x = tok(prompt, truncation=True, padding="max_length", max_length=512)
    y = tok(ex["output"], truncation=True, padding="max_length", max_length=512)
    x["labels"] = y["input_ids"]
    return x

In [18]:
from datasets import load_from_disk

ds = load_from_disk("preprocessed_dataset")

In [21]:
tds = ds["train"].map(pack, remove_columns=ds["train"].column_names)
vds = ds["test"].map(pack, remove_columns=ds["test"].column_names)

Map:   0%|          | 0/269 [00:00<?, ? examples/s]

Map:   0%|          | 0/68 [00:00<?, ? examples/s]

In [22]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [25]:
gpu_available = torch.cuda.is_available()
bf16_support = gpu_available and torch.cuda.is_bf16_supported()

In [27]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    bf16=bf16_support,
    fp16=not bf16_support and gpu_available
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tds,
    eval_dataset=vds,
    tokenizer=tok,
)

/tmp/ipython-input-1136035130.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [28]:
trainer.train()

Step,Training Loss
10,10.631300
20,10.781900
30,10.742700
40,10.439700
50,10.471700
60,10.436500
70,10.410800
80,10.160000
90,9.949200
100,9.520200


TrainOutput(global_step=204, training_loss=8.08735210755292, metrics={'train_runtime': 58.3553, 'train_samples_per_second': 13.829, 'train_steps_per_second': 3.496, 'total_flos': 210862669824000.0, 'train_loss': 8.08735210755292, 'epoch': 3.0})

In [30]:
trainer.save_model("model_output")
tok.save_pretrained("model_output")

('model_output/tokenizer_config.json',
 'model_output/special_tokens_map.json',
 'model_output/vocab.json',
 'model_output/merges.txt',
 'model_output/added_tokens.json',
 'model_output/tokenizer.json')

In [29]:
ds = load_from_disk("preprocessed_dataset")['test']

In [31]:
from transformers import pipeline

tok = AutoTokenizer.from_pretrained("model_output")
mdl = AutoModelForCausalLM.from_pretrained("model_output")
gen = pipeline("text-generation", model=mdl, tokenizer=tok, device=0)

Device set to use cuda:0


In [33]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'rouge_score' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'rouge_score'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=26feb6a8d6d425f7df05a2626c4c8176928205104e8d9e6e16fa55df2f33ac43
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [34]:
import evaluate

rouge = evaluate.load("rouge")

In [35]:
from tqdm import tqdm

preds, refs = [], []
sample = min(50, len(ds))
for ex in tqdm(ds.select(range(sample))):
    prompt = f"### Instruction: {ex['instruction']}\n### Input: {ex.get('input','')}\n### Output:"
    out = gen(prompt, max_new_tokens=128, do_sample=False)[0]["generated_text"]
    preds.append(out)
    refs.append(ex["output"])

100%|██████████| 50/50 [00:17<00:00,  2.82it/s]


In [36]:
res = rouge.compute(predictions=preds, references=refs)
print("Eval:", res)

Eval: {'rouge1': np.float64(0.28830857504231744), 'rouge2': np.float64(0.1719017486089538), 'rougeL': np.float64(0.23584533371082184), 'rougeLsum': np.float64(0.25508438332811)}
